In [1]:
import keras
import numpy as np
import os

C:\Users\admin\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
conv_base = VGG16(include_top=False, input_shape=(240,320,3))

In [4]:
base_dir = 'imgs'
train_dir = os.path.join(base_dir,'train')
test_dir = os.path.join(base_dir,'test')

In [5]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [6]:
def extract_features(dir, sample_count):
    features = np.zeros((sample_count,7,10,512))
    labels = np.zeros(sample_count)
    generator = datagen.flow_from_directory(
        dir, 
        target_size=(240,320),
        batch_size=batch_size,class_mode='sparse')
    i = 0
    for inputs_batch, labels_batch in generator:
        feature_batch = conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1)*batch_size] = feature_batch
        labels[i*batch_size:(i+1)*batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels

In [7]:
n_train = 1809
n_test = 451

In [8]:
train_features, train_labels = extract_features(train_dir, n_train)
test_features, test_labels = extract_features(test_dir, n_test)

Found 1809 images belonging to 4 classes.
Found 451 images belonging to 4 classes.


In [11]:
np.savez('extracted_data.npz',
         train_features=train_features,
         train_labels=train_labels,
         test_features=test_features,
         test_labels=test_labels)